In [1]:
from xml.dom.minidom import parse
import xml.dom.minidom
import math
import copy
import pyproj
import pandas as pd

In [2]:
class Road:
    def __init__(self,roadType,points,name,start,end):
        self.roadType=roadType
        self.points=points
        self.name=name
        self.start=start
        self.end=end
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

        
class Point:
    def __init__(self,lat,lon):
        self.lat=lat
        self.lon=lon

In [3]:
#OpenStreetMap data of Shenzhen
DOMTree = xml.dom.minidom.parse("/Users/SarahDai/Downloads/map_Shenzhen.xml")
collection = DOMTree.documentElement
if collection.hasAttribute("shelf"):
    print("Root element : %s" % collection.getAttribute("shelf"))

In [4]:
#Get the key elements for analysis
ways = collection.getElementsByTagName("way")
nodes = collection.getElementsByTagName("node")
relations = collection.getElementsByTagName("relation")

In [5]:
points = {}
for node in nodes:
    ref=node.getAttribute("id")
    lat=float(node.getAttribute("lat"))
    lon=float(node.getAttribute("lon"))
    point=Point(lat,lon)
    points[ref]=point

In [6]:
#Link nodes information to each road
roads={}
i=0
for way in ways:
    tags=way.getElementsByTagName("tag")
    reff=way.getAttribute("id")
    if len(tags)>0:
        roadType=""
        roadName=""
        for tag in tags:
            if tag.getAttribute("k")=="highway":
                roadType=tag.getAttribute("v")
                continue
            if tag.getAttribute("k")=="name":
                roadName=tag.getAttribute("v")
                continue
        if roadType in ["motorway","trunk","primary","secondary","tertiary","unclassified","residential","service"]:
            pos=way.getElementsByTagName("nd")
            roadPoints=[]
            start=points[pos[0].getAttribute("ref")]
            end=points[pos[len(pos)-1].getAttribute("ref")]
            for po in pos:
                ref=po.getAttribute("ref")
                roadPoints.append(points[ref])
            road=Road(roadType,roadPoints,roadName,start,end)
            roads[reff]=road

In [7]:
roads['310026884'].name

''

In [8]:
#link the seprate roads into major roads
for relation in relations:
    ta=relation.getElementsByTagName("tag")
    for t in ta:
        if t.getAttribute("k")=="route":
            roadType=t.getAttribute("v")
            continue
    if roadType=="road":
        tags=relation.getElementsByTagName("member")
        if len(tags)>0: 
            for tag in tags:
                if tag.getAttribute("ref") in roads:
                    for ta in tags:
                        if ta.getAttribute("ref") in roads:
                            roadtag=roads[tag.getAttribute("ref")]
                            roadta=roads[ta.getAttribute("ref")]
                            if (roadtag.end.lat==roadta.start.lat) and (roadtag.end.lon==roadta.start.lon):
                                roads[tag.getAttribute("ref")].points.extend(roads[ta.getAttribute("ref")].points)
                                roads[tag.getAttribute("ref")].end=roads[ta.getAttribute("ref")].end
                                del roads[ta.getAttribute("ref")]

In [9]:
len(roads)

21268

In [10]:
tag.getAttribute("ref")

'328234603'

In [11]:
roads['332001967']

In [12]:
#For split the roads into paths less than 1 mile
def repoint(points):
    point=points[0]
    pp=[point]
    newPoints=[]
    gdist=0
    for i in range(1,len(points)):
        tempPoint=points[i]
        distt=dist(point,tempPoint)
        if distt<=(1-gdist):
            pp.append(tempPoint)
            gdist=gdist+dist(point,tempPoint)
            point=tempPoint
        if distt>(1-gdist):
            while dist(point,tempPoint)>(1-gdist):
                endpoint=newEnd(point,tempPoint,1-gdist)
                pp.append(endpoint)
                point=endpoint
                newPoints.append(pp)
                pp=[point]
                gdist=0
            if dist(point,tempPoint)<=(1-gdist):
                gdist=dist(point,tempPoint)
                pp.append(tempPoint)
                point=tempPoint
    newPoints.append(pp)
    return newPoints

In [13]:
def dist(pointa,pointb):
    radlat1=math.radians(pointa.lat)
    radlat2=math.radians(pointb.lat)
    a=radlat1-radlat2
    b=math.radians(pointa.lon)-math.radians(pointb.lon)
    s=2*math.asin(math.sqrt(math.pow(math.sin(a/2),2)+math.cos(radlat1)*math.cos(radlat2)*math.pow(math.sin(b/2),2)))
    earth_radius=3958.7583
    s=s*earth_radius
    if s<0:
        return -s
    else:
        return s

In [14]:
def newEnd(pointa,pointb,dist):
    tempd=dist/100/0.6213712
    if abs(pointb.lat-pointa.lat)<=0.000000001:
        return Point(pointa.lat,pointa.lon+tempd)
    t1=math.sqrt(tempd*tempd/(1+(pointb.lon-pointa.lon)**2/(pointb.lat-pointa.lat)**2))
    if pointb.lat<pointa.lat:
        t1=0-t1
    t2=(pointb.lon-pointa.lon)/(pointb.lat-pointa.lat)*t1
    result=Point(t1+pointa.lat,t2+pointa.lon)
    return result

In [15]:
newroads=copy.deepcopy(roads)

In [16]:
#Split the roads into one-mile-long paths
for road in newroads:
    newroads[road].points=repoint(roads[road].points)

In [16]:
with open('/Users/SarahDai/PycharmProjects/untitled1/points.txt', 'w') as f:
    for road in newroads:
        outputString=newroads[road].roadType;
        for points in newroads[road].points:
            for i in range(0,len(points)-1):
                output=outputString+", "+str(points[i].lat)+", "+str(points[i].lon)+", "+str(points[i+1].lat)+", "+str(points[i+1].lon);
                f.write(output+"\n");

In [17]:
len(newroads)
print(roads['315897135'].points)
print(newroads['315897135'].points[0])

[<__main__.Point object at 0x200d5c518>, <__main__.Point object at 0x200d5c400>]
[<__main__.Point object at 0x200d5c518>, <__main__.Point object at 0x200d5c400>]


In [18]:
print (newroads[road].points)

[[<__main__.Point object at 0x200be2630>, <__main__.Point object at 0x200bdfdd8>, <__main__.Point object at 0x200bdfcc0>, <__main__.Point object at 0x200be2400>, <__main__.Point object at 0x200be5b00>, <__main__.Point object at 0x200be50b8>, <__main__.Point object at 0x200be29e8>, <__main__.Point object at 0x200be22e8>, <__main__.Point object at 0x200be5630>, <__main__.Point object at 0x200be5ba8>, <__main__.Point object at 0x200be5198>, <__main__.Point object at 0x200be5908>]]


In [19]:
#Load in the taxi data
with open('/Users/SarahDai/Downloads/TaxiData.txt', 'r') as f:
    content=f.read();

In [20]:
print(len(content.split("\n")))

46927856


In [21]:
import sklearn.utils as sk

In [27]:
content_array = sk.resample(content.split("\n"),replace=False,n_samples=200000)

In [28]:
with open("/Users/SarahDai/PycharmProjects/untitled1/newsplit.txt","w") as f:
    for con in content_array:
        f.write(con+"\n")

In [1]:
import pandas as pd
taxi_points =pd.read_csv('/Users/SarahDai/PycharmProjects/untitled1/newsplit.csv')

In [2]:
taxis=[]
for i in range(len(taxi_points)):
    taxi={}
    taxi_point=taxi_points.iloc[i]
    taxi["lat"]=taxi_point["lat"]
    taxi["lon"]=taxi_point["lon"]
    taxis.append(taxi)

In [18]:
len(taxis)

200000

In [19]:
print(taxis[1])

{'lat': 22.637167000000002, 'lon': 114.17728400000001}


In [20]:
sortedIncidents=sorted(taxis,key=lambda d:d['lon'])

In [21]:
len(sortedIncidents)

200000

In [22]:
def findPossibleLon(start,end,dist):
    if start.lon>end.lon:
        emp=start
        start=end
        end=emp
    dist=dist*0.0001894/100/0.6213712
    starti=0
    inc=[]
    while (starti<len(sortedIncidents)) and (sortedIncidents[starti]["lon"]<start.lon):
        starti=starti+1
    while (starti<len(sortedIncidents)) and (sortedIncidents[starti]["lon"]<=end.lon):
        if belongToRoad(sortedIncidents[starti],start,end,dist):
            inc.append(sortedIncidents[starti])
        starti+=1
    return inc

In [23]:
def belongToRoad(point,start,end,dist):
    if (end.lat-start.lat)<0.00000001:
        if ((point["lon"]<=end.lon) and (point["lon"]>=start.lon)):
            if (abs(point["lat"]-start.lat)<dist):
                return True
    else:
        if ((point["lon"]-start.lon)>((start.lon-end.lon)/(end.lat-start.lat)*(point["lat"]-start.lat))):
            if ((point["lon"]-end.lon)<((start.lon-end.lon)/(end.lat-start.lat)*(point["lat"]-end.lat))):
                a=end.lon-start.lon
                b=end.lat-start.lon
                down=math.sqrt(a**2+b**2)
                up=a*point["lat"]-b*point["lon"]-b*start.lat+a*start.lon
                if ((up/down)<dist):
                    return True
    return False

In [37]:
roadsToDraw=[]
not0count=0
totalcount=0
time=0
for ro in newroads:
    road=newroads[ro]
    roadToDraw={}
    psOfRoad=[]
    if (time%1000==0):
        print(time)
    time+=1
    roadToDraw["type"]=road.roadType
    for path in road.points:
        pat={}
        po={}
        startPoint=path[0]
        po["lat"]=startPoint.lat
        po["lon"]=startPoint.lon
        pos=[po]
        count=0
        incs=[]
        for i in range(1,len(path)):
            po1={}
            nowPoint=path[i] 
            inc=findPossibleLon(startPoint,nowPoint,dist=100)
            count=count+len(inc)
            incs.extend(inc)
            startPoint=nowPoint
            po1["lat"]=startPoint.lat
            po1["lon"]=startPoint.lon
            pos.append(po1)
        if (count>0):
            not0count+=1
            totalcount+=count
        pat["count"]=count
        pat["points"]=pos
        psOfRoad.append(pat)
    roadToDraw["path"]=psOfRoad
    roadsToDraw.append(roadToDraw)

In [54]:
roadsToDraw[0]

In [25]:
len(roadsToDraw)

4694

In [60]:
import json
roadsToDraw=
result = json.dumps(roadsToDraw)
json.dump(roadsToDraw,open("/Users/SarahDai/PycharmProjects/untitled1/output_taxi.json","w"))